In [2]:
from cp import cp

In [16]:
def preprocess_inventory_from_list(lines, data):
    ingredients = data["all_ingredients"]
    inventory = dict()
    for ingrli in ingredients:
        inventory[ingrli["i_id"]] = {"name": ingrli["name"], "amount": 0}
    for line in lines:
        ingr = line.strip().split(": ")
        for ingrli in ingredients:
            if ingr[0] == ingrli["name"]:
                inventory[ingrli["i_id"]]["amount"] = float(ingr[1])
    return inventory

def preprocess_disliked_from_list(lines, data):
    ingredients = data["all_ingredients"]
    disliked = dict()
    for line in lines:
        ingr = line.strip()
        for ingrli in ingredients:
            if ingr == ingrli["name"]:
                disliked[ingrli["name"]] = ingrli["i_id"]
    return disliked


In [18]:
def process_disliked_ingredients(data, disliked_ingredients):
    disliked_ct = []
    for recipe in data["recipes"]:
        count = 0
        for ing in recipe["ingredients"]:
            ing_id = ing["i_id"]
            ing_name = data["all_ingredients"][ing_id]["name"]
            if ing_name in disliked_ingredients:
                count += 1
        disliked_ct.append(count)
    return disliked_ct

TEST 1: Simple Test

This test only has 2 ingredients which the user already owns, and 1 recipe using the ingredients, so the solver will just pick the 1 recipe.

In [ ]:
data1 = {
    "all_ingredients": [
        {"i_id": 0, "name": "Chicken Breast", "unit_price": 100},
        {"i_id": 1, "name": "Rice", "unit_price": 50},
    ],
    "recipes": [
        {
            "name": "Chicken and Rice",
            "ingredients": [{"i_id": 0, "name": "Chicken Breast", "proportion": 1.0},
                            {"i_id": 1, "name": "Rice", "proportion": 1.0}],
            "nutrients": {"calories": 400, "protein": 40, "cholesterol": 80}
        }
    ]
}

budget1 = 10 
calorie_cap1 = 500
inventory1 = [
    {"i_id": 0, "amount": 1.0}, 
    {"i_id": 1, "amount": 1.0},  
]
disliked_ct1 = [0]  # no dislikes

cp(data1, budget1, calorie_cap1, inventory1, disliked_ct1, chosen_meals=1)


Solution status: OPTIMAL
Chosen recipe IDs: [0]
  -> Chicken and Rice
Extra purchased amounts:
Total cost used: 0.0 (Budget = 10)
Total Protein: 40
Total Calories: 400
Total Cholestrol: 80
Calories to Protein Ratio: 10.0
Calories to Cholesterol Ratio: 5.0


In [5]:
# No starting inventory, must buy all ingredients

data2 = {
    "all_ingredients": [
        {"i_id": 0, "name": "Beef", "unit_price": 200},
        {"i_id": 1, "name": "Potatoes", "unit_price": 75},
    ],
    "recipes": [
        {
            "name": "Beef Stew",
            "ingredients": [{"i_id": 0, "name": "Beef", "proportion": 1.5},
                            {"i_id": 1, "name": "Potatoes", "proportion": 2.0}],
            "nutrients": {"calories": 900, "protein": 50, "cholesterol": 120}
        }
    ]
}

budget2 = 20
calorie_cap2 = 1000
inventory2 = [
    {"i_id": 0, "amount": 0.0}, 
    {"i_id": 1, "amount": 0.0},
]
disliked_ct2 = [0]

cp(data2, budget2, calorie_cap2, inventory2, disliked_ct2, chosen_meals=1)


Solution status: OPTIMAL
Chosen recipe IDs: [0]
  -> Beef Stew
Extra purchased amounts:
Ingredient 0 (Beef): 2.0
Ingredient 1 (Potatoes): 2.0
Total cost used: 5.5 (Budget = 20)
Total Protein: 50
Total Calories: 900
Total Cholestrol: 120
Calories to Protein Ratio: 18.0
Calories to Cholesterol Ratio: 7.5


In [7]:
# One recipe is highly disliked

data3 = {
    "all_ingredients": [
        {"i_id": 0, "name": "Salmon", "unit_price": 300},
        {"i_id": 1, "name": "Asparagus", "unit_price": 150},
    ],
    "recipes": [
        {
            "name": "Salmon Dinner",
            "ingredients": [{"i_id": 0, "name": "Salmon", "proportion": 1.0},
                            {"i_id": 1, "name": "Asparagus", "proportion": 0.5}],
            "nutrients": {"calories": 700, "protein": 45, "cholesterol": 90}
        }
    ]
}

budget3 = 15
calorie_cap3 = 800
inventory3 = [
    {"i_id": 0, "amount": 1.0},
    {"i_id": 1, "amount": 1.0},
]
disliked_ct3 = [5]  # Very disliked

cp(data3, budget3, calorie_cap3, inventory3, disliked_ct3, chosen_meals=1)


Solution status: OPTIMAL
Chosen recipe IDs: [0]
  -> Salmon Dinner
Extra purchased amounts:
Total cost used: 0.0 (Budget = 15)
Total Protein: 45
Total Calories: 700
Total Cholestrol: 90
Calories to Protein Ratio: 15.555555555555555
Calories to Cholesterol Ratio: 7.777777777777778


In [ ]:
# 3 recipes, only 2 can be chosen

data4 = {
    "all_ingredients": [
        {"i_id": 0, "name": "Chicken", "unit_price": 100},
        {"i_id": 1, "name": "Pasta", "unit_price": 75},
        {"i_id": 2, "name": "Spinach", "unit_price": 125},
    ],
    "recipes": [
        {
            "name": "Chicken Pasta",
            "ingredients": [{"i_id": 0, "name": "Chicken", "proportion": 1.0},
                            {"i_id": 1, "name": "Pasta", "proportion": 1.0}],
            "nutrients": {"calories": 800, "protein": 50, "cholesterol": 100}
        },
        {
            "name": "Spinach Salad",
            "ingredients": [{"i_id": 2, "name": "Spinach", "proportion": 1.0}],
            "nutrients": {"calories": 200, "protein": 15, "cholesterol": 0}
        },
        {
            "name": "Pasta Salad",
            "ingredients": [{"i_id": 1, "name": "Pasta", "proportion": 0.5},
                            {"i_id": 2, "name": "Spinach", "proportion": 0.5}],
            "nutrients": {"calories": 400, "protein": 20, "cholesterol": 10}
        }
    ]
}

budget4 = 20
calorie_cap4 = 900
inventory4 = [
    {"i_id": 0, "amount": 1.0},
    {"i_id": 1, "amount": 1.0},
    {"i_id": 2, "amount": 1.0},
]
disliked_ct4 = [0, 0, 0]

cp(data4, budget4, calorie_cap4, inventory4, disliked_ct4, chosen_meals=2)


Solution status: OPTIMAL
Chosen recipe IDs: [0, 1]
  -> Chicken Pasta
  -> Spinach Salad
Extra purchased amounts:
Ingredient 1 (Pasta): 1.0
Ingredient 2 (Spinach): 1.0
Total cost used: 2.0 (Budget = 20)
Total Protein: 65
Total Calories: 1000
Total Cholestrol: 100
Calories to Protein Ratio: 15.384615384615385
Calories to Cholesterol Ratio: 10.0


In [21]:
data4 = {
    "all_ingredients": [
        {"i_id": 0, "name": "Chicken", "unit_price": 100},
        {"i_id": 1, "name": "Pasta", "unit_price": 75},
        {"i_id": 2, "name": "Spinach", "unit_price": 125},
    ],
    "recipes": [
        {
            "name": "Chicken Pasta",
            "ingredients": [{"i_id": 0, "proportion": 1.0},
                            {"i_id": 1, "proportion": 1.0}],
            "nutrients": {"calories": 800, "protein": 50, "cholesterol": 100}
        },
        {
            "name": "Spinach Salad",
            "ingredients": [{"i_id": 2, "proportion": 1.0}],
            "nutrients": {"calories": 200, "protein": 15, "cholesterol": 0}
        },
        {
            "name": "Pasta Salad",
            "ingredients": [{"i_id": 1, "proportion": 0.5},
                            {"i_id": 2, "proportion": 0.5}],
            "nutrients": {"calories": 200, "protein": 20, "cholesterol": 10}
        }
    ]
}

budget4 = 20
calorie_cap4 = 900
inventory4_lines = [
    "Chicken: 1",
    "Pasta: 1",
    "Spinach: 1"
]

# Dislike Chicken
disliked4_lines = ["Spinach"]

# Preprocess inventory and disliked ingredients
inventory4 = preprocess_inventory_from_list(inventory4_lines, data4)
disliked4 = preprocess_disliked_from_list(disliked4_lines, data4)
disliked_ct4 = process_disliked_ingredients(data4, disliked4)

# Running the CP solver with 2 recipes to be chosen, avoiding Chicken
cp(data4, budget=budget4, calorie_cap=calorie_cap4, inventory=inventory4, disliked_ct=disliked_ct4, chosen_meals=2)


Solution status: OPTIMAL
Chosen recipe IDs: [0, 2]
  -> Chicken Pasta
  -> Pasta Salad
Extra purchased amounts:
Ingredient 1 (Pasta): 1.0
Ingredient 2 (Spinach): 1.0
Total cost used: 2.0 (Budget = 20)
Total Protein: 70
Total Calories: 1000
Total Cholestrol: 110
Calories to Protein Ratio: 14.285714285714286
Calories to Cholesterol Ratio: 9.090909090909092


In [39]:
# Impossible to satisfy calorie cap

data5 = {
    "all_ingredients": [
        {"i_id": 0, "name": "Steak", "unit_price": 500},
    ],
    "recipes": [
        {
            "name": "Steak Dinner",
            "ingredients": [{"i_id": 0, "name": "Steak", "proportion": 1.0}],
            "nutrients": {"calories": 1200, "protein": 80, "cholesterol": 150}
        }
    ]
}

budget5 = 10
calorie_cap5 = 800  # Way too low for this steak recipe
inventory5 = [
    {"i_id": 0, "amount": 1.0},
]
disliked_ct5 = [0]

cp(data5, budget5, calorie_cap5, inventory5, disliked_ct5, chosen_meals=1)


No feasible solution found.


In [23]:
data_large = {
    "all_ingredients": [
        {"i_id": 0, "name": "chicken", "unit_price": 150},
        {"i_id": 1, "name": "rice", "unit_price": 80},
        {"i_id": 2, "name": "broccoli", "unit_price": 120},
        {"i_id": 3, "name": "beef", "unit_price": 200},
        {"i_id": 4, "name": "pasta", "unit_price": 90},
        {"i_id": 5, "name": "tomato", "unit_price": 100},
        {"i_id": 6, "name": "cheese", "unit_price": 180},
        {"i_id": 7, "name": "lettuce", "unit_price": 110},
        {"i_id": 8, "name": "onion", "unit_price": 70},
        {"i_id": 9, "name": "beans", "unit_price": 130},
        {"i_id": 10, "name": "potato", "unit_price": 60},
    ],
    "recipes": [
        {"name": "chicken stir fry", "ingredients": [{"i_id": 0, "proportion": 0.5}, {"i_id": 2, "proportion": 0.3}], "nutrients": {"calories": 500, "protein": 40, "cholesterol": 50}},
        {"name": "beef stew", "ingredients": [{"i_id": 3, "proportion": 0.6}, {"i_id": 10, "proportion": 0.4}], "nutrients": {"calories": 700, "protein": 60, "cholesterol": 90}},
        {"name": "veggie pasta", "ingredients": [{"i_id": 4, "proportion": 0.5}, {"i_id": 5, "proportion": 0.3}, {"i_id": 2, "proportion": 0.2}], "nutrients": {"calories": 600, "protein": 20, "cholesterol": 10}},
        {"name": "chili beans", "ingredients": [{"i_id": 9, "proportion": 0.7}, {"i_id": 8, "proportion": 0.2}], "nutrients": {"calories": 500, "protein": 25, "cholesterol": 5}},
        {"name": "chicken salad", "ingredients": [{"i_id": 0, "proportion": 0.4}, {"i_id": 7, "proportion": 0.3}], "nutrients": {"calories": 350, "protein": 30, "cholesterol": 40}},
        {"name": "cheesy pasta", "ingredients": [{"i_id": 4, "proportion": 0.5}, {"i_id": 6, "proportion": 0.5}], "nutrients": {"calories": 650, "protein": 25, "cholesterol": 60}},
        {"name": "baked potato", "ingredients": [{"i_id": 10, "proportion": 0.9}], "nutrients": {"calories": 400, "protein": 8, "cholesterol": 0}},
        {"name": "beef tacos", "ingredients": [{"i_id": 3, "proportion": 0.5}, {"i_id": 8, "proportion": 0.3}], "nutrients": {"calories": 600, "protein": 45, "cholesterol": 70}},
        {"name": "rice and beans", "ingredients": [{"i_id": 1, "proportion": 0.4}, {"i_id": 9, "proportion": 0.4}], "nutrients": {"calories": 550, "protein": 18, "cholesterol": 5}},
        {"name": "pasta primavera", "ingredients": [{"i_id": 4, "proportion": 0.6}, {"i_id": 5, "proportion": 0.3}, {"i_id": 7, "proportion": 0.1}], "nutrients": {"calories": 620, "protein": 22, "cholesterol": 10}},
    ]
}

inventory_lines_large = [
    "chicken: 2",
    "rice: 5",
    "broccoli: 1",
    "beef: 2",
    "pasta: 4",
    "tomato: 3",
    "cheese: 2",
    "lettuce: 2",
    "onion: 3",
    "beans: 4",
    "potato: 5",
]

disliked_lines_large = [
    "broccoli", "onion"
]

inventory_large = preprocess_inventory_from_list(inventory_lines_large, data_large)
disliked_large = preprocess_disliked_from_list(disliked_lines_large, data_large)
disliked_ct_large = process_disliked_ingredients(data_large, disliked_large)

# run
cp(data_large, budget=50, calorie_cap=800, inventory=inventory_large, disliked_ct=disliked_ct_large, chosen_meals=5)


Solution status: OPTIMAL
Chosen recipe IDs: [1, 4, 5, 8, 9]
  -> beef stew
  -> chicken salad
  -> cheesy pasta
  -> rice and beans
  -> pasta primavera
Extra purchased amounts:
Total cost used: 0.0 (Budget = 50)
Total Protein: 155
Total Calories: 2870
Total Cholestrol: 205
Calories to Protein Ratio: 18.516129032258064
Calories to Cholesterol Ratio: 14.0
